In [1]:
!pip install autokeras

     |████████████████████████████████| 174kB 5.7MB/s 
     |████████████████████████████████| 102kB 8.3MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-cp37-none-any.whl size=9569 sha256=58ba8a14997b30b16eb80d53371a19951c9df4f199893025eadcaec469bfe3cf
  Stored in directory: /root/.cache/pip/wheels/95/99/a8/6b116b0f69be60cc475d1cd36680f3a09f284d86655bb99d93
Successfully built kt-legacy


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model

import autokeras as ak

You can easily export your model the best model found by AutoKeras as a Keras Model.

The following example uses [ImageClassifier](/image_classifier) as an example.
All the tasks and the [AutoModel](/auto_model/#automodel-class) has this
[export_model](/auto_model/#export_model-method) function.


In [3]:
print(tf.__version__)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# # Initialize the image classifier.
# clf = ak.ImageClassifier(
#     overwrite=True, max_trials=5, objective="val_accuracy" # The max_trials refer to how many different models will be attempted. 
# )  # Try only 1 model.(Increase accordingly)
# # Feed the image classifier with training data.
# clf.fit(x_train, y_train, time_limit=12 * 60 * 6, epochs=1) # Change no of epochs to improve the model #20

clf = ak.ImageClassifier(overwrite=True, max_trials=1, objective="val_accuracy")
clf.fit(x_train, y_train, epochs=1, max_queue_size=2) # 1 Hour
# clf.final_fit(x_train, y_train, x_test, y_test, retrain=False, trainer_args={'max_no_improvement_num': 3})

# # Export as a Keras Model.
model = clf.export_model()

print(type(model))  # <class 'tensorflow.python.keras.engine.training.Model'>

try:
    model.save("model_autokeras", save_format="tf")
except Exception:
    model.save("model_autokeras.h5")


loaded_model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))
print(predicted_y)


Trial 1 Complete [00h 02m 25s]
val_accuracy: 0.9779166579246521

Best val_accuracy So Far: 0.9779166579246521
Total elapsed time: 00h 02m 25s
INFO:tensorflow:Oracle triggered exit
1875/1875 [==============================] - 172s 91ms/step - loss: 0.1539 - accuracy: 0.9528
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets
<class 'tensorflow.python.keras.engine.functional.Functional'>
INFO:tensorflow:Assets written to: model_autokeras/assets
[[5.5174358e-09 1.9316068e-10 3.9156227e-08 ... 9.9999964e-01
  2.8286120e-09 2.3889510e-07]
 [1.2987198e-05 1.0993873e-05 9.9988019e-01 ... 1.3601076e-12
  2.3652528e-06 6.0710999e-11]
 [4.2812746e-07 9.9918419e-01 8.2734608e-05 ... 1.9262019e-04
  1.3482567e-05 1.0661159e-06]
 ...
 [9.8110978e-11 2.4243988e-08 7.8914458e-10 ... 9.0053563e-06
  1.1297294e-05 2.5448602e-05]
 [1.0387388e-06 8.9129193e-09 5.4987332e-08 ... 1.5406783e-08
  1.4192591e-03 1.2909418e-07]
 [4.3079154e-07 3.3988068e-10 3.5031525e-07 ... 2.9498425e-11
 

In [4]:
model.save("model_autokeras.h5")

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
cast_to_float32 (CastToFloat (None, 28, 28)            0         
_________________________________________________________________
expand_last_dim (ExpandLastD (None, 28, 28, 1)         0         
_________________________________________________________________
normalization (Normalization (None, 28, 28, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0     

In [6]:
loaded_model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)

In [7]:
predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))
print(predicted_y)

[[5.5174358e-09 1.9316068e-10 3.9156227e-08 ... 9.9999964e-01
  2.8286120e-09 2.3889510e-07]
 [1.2987198e-05 1.0993873e-05 9.9988019e-01 ... 1.3601076e-12
  2.3652528e-06 6.0710999e-11]
 [4.2812746e-07 9.9918419e-01 8.2734608e-05 ... 1.9262019e-04
  1.3482567e-05 1.0661159e-06]
 ...
 [9.8110978e-11 2.4243988e-08 7.8914458e-10 ... 9.0053563e-06
  1.1297294e-05 2.5448602e-05]
 [1.0387388e-06 8.9129193e-09 5.4987332e-08 ... 1.5406783e-08
  1.4192591e-03 1.2909418e-07]
 [4.3079154e-07 3.3988068e-10 3.5031525e-07 ... 2.9498425e-11
  3.9570844e-07 1.2010734e-10]]


In [8]:
clf.evaluate(x_test, y_test)

313/313 [==============================] - 7s 23ms/step - loss: 0.0581 - accuracy: 0.9801


[0.058103613555431366, 0.9800999760627747]

In [9]:
score = clf.evaluate(x_test, y_test)
print("Training loss", score[0])
print("Training accuracy", score[1])

313/313 [==============================] - 7s 22ms/step - loss: 0.0581 - accuracy: 0.9801
Training loss 0.058103613555431366
Training accuracy 0.9800999760627747
